In [1]:
import random
import json
import os
from apache_atlas.utils.Constants import EndRelations, TypeNames
from apache_atlas.utils.API import API, HTTPMethod
from apache_atlas.client.ApacheAtlas import ApacheAtlasClient

atlas_section = ApacheAtlasClient(
    "http://10.100.100.61:21000",
    "admin",
    "admin"
)

### Criação das Tabelas

In [ ]:
# Executado...

groups_sia ={
    #'AB': 'APAC de Cirurgia Bariátrica',
    'ABO': 'APAC de Acompanhamento Pós Cirurgia Bariátrica',
    'ACF': 'APAC de Confecção de Fístula',
    'AD': 'APAC de Laudos Diversos',
    'AM': 'APAC de Medicamentos',
    'AMP': 'APAC de Acompanhamento Multiprofissional',
    'AN': 'APAC de Nefrologia',
    'AQ': 'APAC de Quimioterapia',
    'AR': 'APAC de Radioterapia',
    'ATD': 'APAC de Tratamento Dialítico',
    'BI': 'Boletim de Produção Ambulatorial individualizado',
    'PA': 'Produção Ambulatorial',
    'PS': 'RAAS Psicossocial',
    'SAD': 'RAAS de Atenção Domiciliar'
}

for acronymus, name in groups_sia.items():
    atlas_section.entity.create_entity_dt_table({
       'name': f"{name} ({acronymus})",
       'acronymus': acronymus,
       'description': f'Tabela de {name} da SIA' 
    }, 'SIA')

### Criação de Colunas

In [ ]:
# Executado...

import json
import os

pasta = 'dicionarios'

for arquivo in os.listdir(pasta):
    sigla = arquivo.split('-')[1].split('.')[0]
    caminho_dict = os.path.join(pasta, arquivo)

    atlas_section.entity.create_entity_columns(caminho_dict, sigla)

### Criação de Tabelas Anuais/Mensais

In [ ]:
# Executado...

atlas_section.type.create_type([
   {
      "name": TypeNames.MONTLY_TABLE,
      "description": "Representa uma tabela em um mês específico",
      "superTypes": [TypeNames.ANUAL_TABLE],
      "attributeDefs": [
        {
          "name": "month",
          "typeName": "int",
          "isOptional": True,
          "isUnique": False,
          "isIndexable": False
        }
      ]
    },
])

atlas_section.type.create_type(relationshipDefs=[
    {
    "name": "rsh_table_to_columns_time",
    "relationshipCategory": "AGGREGATION",
    "propagateTags": "NONE",
    "endDef1": {
        "type": TypeNames.TABLE,
        "name": EndRelations.END_TABLE_TO_COLUMNS_TIME[0],
        "cardinality": "SET",
        "isContainer": True,
        "isLegacyAttribute": False
    },
    "endDef2": {
        "type": TypeNames.ANUAL_TABLE,
        "name": EndRelations.END_TABLE_TO_COLUMNS_TIME[1],
        "cardinality": "SINGLE",
        "isContainer": False,
        "isLegacyAttribute": False
    }
}, 
])


In [ ]:
# Executado ...

url = atlas_section.search.SEARCH_BY_TYPENAME.format_path({
    'typeName': TypeNames.ANUAL_TABLE
})

response = atlas_section.request(url)

filters = list(filter(
    lambda x: x['attributes']['name'].startswith("DO"),
    response['entities']
)) 

table = atlas_section.search.search_table_by_acronymus("DO")

for i in filters:
    i['attributes'][EndRelations.END_TABLE_TO_COLUMNS_TIME[1]] = {
        'guid': table['guid']
    }

atlas_section.entity.create_multiple_entities(filters)

In [ ]:
# Executado apenas para o SIA
# Deu problema no AM (AM Altura), AR sei lá oq

import os
import json

pasta = 'data'

for arquivo in os.listdir(pasta):
    if arquivo.endswith('sia.json'):
        caminho_arquivo = os.path.join(pasta, arquivo)

        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            dados_json = json.load(f)
        
        for acronymus, lineage in dados_json.items():
            atlas_section.lineage.create_lineage_table(lineage, acronymus)

http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_table&attrName=acronymus&attrValuePrefix=PA&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/guid/395ad198-2dfc-43bf-8598-09b5076c504c
http://10.100.100.61:21000/api/atlas/v2/entity/bulk


In [ ]:
# Executado...

atlas_section.entity.create_database_entity({
    'name': 'Database de Tabelas',
    'description': 'Database apenas para agrupar determinadas tabelas (Ignorar)',
    'acronymus': TypeNames.ACRONYMUS_DATABASE_DTC
}, "DataSUS")

atlas_section.entity.create_entity_dt_table({
    "name": "Tabela de Colunas",
    "description": "Tabela de referência para colunas/arquivos alterados por processos (Ignorar)",
    "acronymus": TypeNames.ACRONYMUS_TABLE_DTC
}, f"{TypeNames.ACRONYMUS_DATABASE_DTC}")